## Модели для выбора инструментов инвестирования ПИФ

In [1]:
import pandas as pd
import os

In [2]:
def import_data(file_path): #получение данных
    try:
        df = pd.read_csv(file_path, delimiter=';', encoding='utf-8', na_values=['None'])
        print(f"Импорт данных успешный")
        return df
    except FileNotFoundError:
        print(f"Файл '{file_path}' не найден.")
        return None
    except Exception as e:
        print(f"Произошла ошибка: {e}")
        return None
    
file_path = os.path.join('..', 'data.txt')
df = import_data(file_path)

Импорт данных успешный


In [3]:
def preprocess_data(df): #предобработка данных
    if df is not None and df.isna().sum().any():
        df.dropna(inplace=True)

    def inverse(value):
        return value / -1

    df['risk'] = df['risk'].apply(inverse)
    df['min_contrib'] = df['min_contrib'].apply(inverse)

    columns_to_drop = ['name', 'type']
    df = df.drop(columns=columns_to_drop)

    return df

if df is not None:
    df_math = preprocess_data(df)
    display(df_math) 
    
    res_dict = {'weight_sum': [], 'pareto': [],'scaling': []}

,ID,profit,risk,effect,share_holder,min_contrib
0,11,6.22,-1.035,-0.120,265,-1000.0
1,2,12.45,-0.190,0.330,275,-100000.0
2,3,3.52,0.007,0.482,142,-300000.0
3,4,0.38,-0.776,0.046,80,-10000000.0
4,5,1.77,-1.084,0.007,141,-300000.0
5,12,-2.54,-0.042,-0.117,1,-50000.0
6,6,81.40,-0.699,0.001,6976,-1000.0
7,7,2.93,-1.002,0.132,6887,-1000.0
10,13,-0.72,-0.093,-0.068,44,-1000.0
11,14,2.37,-0.031,0.035,534,-1000.0


In [4]:
def weighted_sum_method(df, priorities): # многокритериальной оптимизации методом взвешенной суммы
    normalized = df.iloc[:, 1:].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
    weighted = normalized * priorities
    df['Total'] = weighted.sum(axis=1)
    
    return df.sort_values(by='Total', ascending=False)

user_priorities = {
    'profit': 1,
    'risk': 2,
    'effect': 3,
    'share_holder': 4,
    'min_contrib': 5
}

weights = [user_priorities[col] for col in df_math.columns[1:]]
result = weighted_sum_method(df_math, weights)
top_5 = result.head(5)
res_dict['weight_sum'] = top_5['ID'].values

filtered_data = df[df['ID'].isin(top_5['ID'].values)]
print('Топ-5 инструментов - Взвешенная сумма:')
display(filtered_data)

Топ-5 инструментов - Взвешенная сумма:


,ID,name,type,profit,risk,effect,share_holder,min_contrib
2,3,Активо двенадцать,Закрытый,3.52,0.007,0.482,142,-300000.0
6,6,Альфа-Капитал Видеоигры,Биржевой,81.40,-0.699,0.001,6976,-1000.0
7,7,Альфа-Капитал Космос,Биржевой,2.93,-1.002,0.132,6887,-1000.0
18,21,ДОХОДЪ. Российские акции. Первый эшелон,Открытый,98.52,0.298,0.177,174,-1000.0
24,27,Мои акции,Открытый,83.40,-0.930,0.434,257,-5000.0


In [5]:
def find_pareto_optimal(df, criteria): #многокритериальная оптимизация методом Парето (Парето-оптимальные решения)
    df['ID'] = df['ID'].astype(int)
    pareto_optimal_solutions = []

    for index, row in df.iterrows():
        is_pareto = True
        for _, comp_row in df.iterrows():
            if all(row[c] <= comp_row[c] for c in criteria) and any(row[c] < comp_row[c] for c in criteria):
                is_pareto = False
                break
        if is_pareto:
            pareto_optimal_solutions.append(row)

    # DataFrame с Парето-оптимальными решениями
    pareto_df = pd.DataFrame(pareto_optimal_solutions)
    return pareto_df

criteria = ['profit', 'risk', 'effect', 'share_holder', 'min_contrib']
pareto = find_pareto_optimal(df_math, criteria)
top_5 = pareto.head(5)
res_dict['pareto'] = top_5['ID'].values.astype(int)

filtered_data = df[df['ID'].isin(top_5['ID'].values)]
print('Топ-5 инструментов - Парето:')
display(filtered_data)

Топ-5 инструментов - Парето:


,ID,name,type,profit,risk,effect,share_holder,min_contrib
1,2,Активо два,Закрытый,12.45,-0.190,0.330,275,-100000.0
2,3,Активо двенадцать,Закрытый,3.52,0.007,0.482,142,-300000.0
6,6,Альфа-Капитал Видеоигры,Биржевой,81.40,-0.699,0.001,6976,-1000.0
7,7,Альфа-Капитал Космос,Биржевой,2.93,-1.002,0.132,6887,-1000.0
11,14,Арсагера - фонд облигаций КР 1.55,Открытый,2.37,-0.031,0.035,534,-1000.0


In [6]:
from sklearn import preprocessing

def scale_data(df):
    ids = df['ID'].values
    df_to_scale = df.drop(columns=['ID'])
    col_names = list(df_to_scale)

    s_scaler = preprocessing.StandardScaler()
    stand = s_scaler.fit_transform(df_to_scale)
    df_stand = pd.DataFrame(stand, columns=col_names)

    df_stand['ID'] = ids.astype(int)
    return df_stand

def calculate_top_5(df):
    selected_columns = ['profit', 'risk', 'effect', 'share_holder', 'min_contrib']
    df['select'] = df[selected_columns].mean(axis=1)
    top_5_results = df.nlargest(5, 'select')
    return top_5_results

df_scaled = scale_data(df_math)
top_5 = calculate_top_5(df_scaled)
res_dict['scaling'] = top_5['ID'].values

filtered_data = df[df['ID'].isin(top_5['ID'].values)]
print('Топ-5 инструментов - Шкалирование:')
display(filtered_data)

Топ-5 инструментов - Шкалирование:


,ID,name,type,profit,risk,effect,share_holder,min_contrib
2,3,Активо двенадцать,Закрытый,3.52,0.007,0.482,142,-300000.0
6,6,Альфа-Капитал Видеоигры,Биржевой,81.40,-0.699,0.001,6976,-1000.0
18,21,ДОХОДЪ. Российские акции. Первый эшелон,Открытый,98.52,0.298,0.177,174,-1000.0
21,24,Лидер - Акции,Открытый,65.12,0.050,0.235,96,-500000.0
24,27,Мои акции,Открытый,83.40,-0.930,0.434,257,-5000.0


In [7]:
print("РЕЗУЛЬТАТ\n")

print(f"Метод Взвешенной суммы -> {res_dict['weight_sum']}")
print(f"Метод Парето -> {res_dict['pareto']}")
print(f"Метод Шкалирования -> {res_dict['scaling']}")


values_count = {}
for value_list in res_dict.values():
    for val in value_list:
        if val in values_count:
            values_count[val] += 1
        else:
            values_count[val] = 1

sorted_values_count = sorted(values_count.items(), key=lambda x: x[1], reverse=True)
print("\nСравнение результатов методов:")
for num, count in sorted_values_count:
    print(f"{count} раз(а)/3  ->  {num}")

РЕЗУЛЬТАТ

Метод Взвешенной суммы -> [ 6  7  3 21 27]
Метод Парето -> [ 2  3  6  7 14]
Метод Шкалирования -> [ 6 21 27 24  3]

Сравнение результатов методов:
3 раз(а)/3  ->  6
3 раз(а)/3  ->  3
2 раз(а)/3  ->  7
2 раз(а)/3  ->  21
2 раз(а)/3  ->  27
1 раз(а)/3  ->  2
1 раз(а)/3  ->  14
1 раз(а)/3  ->  24
